<a href="https://colab.research.google.com/github/SmartDvi/Python_for_Data_science/blob/main/TATA_Publication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object of the Analysis


#### - what is the quantity sold per product

#### - The CMO is interested in viewing the top 10 countries which are generating the highest revenue. Additionallity, the CMO is also interested in viewing the quantity sold with the revenue generated. The CMO does not want to have united kingdom in the visual   

#### - What is the monthly trend of revenue, which months have faced the biggest increase/decrease?

#### - Which months generated the most revenue? Is there a seasonality in sales?

#### - the Number of Customers that reffers a Particular product Description (who are the unique Customers per product)

#### - who are the unique Customers per product and country

#### - The CMU of the online store wants to view the information on the top 10 customers by revenue. He is interested in a visual that shows the greatest revenue generating customer at the statrt and gradually declines to the lower revenue generating customers. the CMO wants to target the higher revenue generating customers and ensure that they remain stisfied with their products.

#### - Who are the top customers and how much do they contribute to the total revenue? Is the business dependent on these customers or is the customer base diversified?

#### - Is there a Particular time or period of the day the business tend to generate more Sale?

#### - the CEO is looking to gain insight on the demand for their products. He wants to look at the all countries and see which regions have the greatest demand for their products. once the Ceo gets an idea of the rgions that has the high demand, he will initiate an expression strategy which will allow the company to target these area and generate more business from this regions. He wants to view the entire data on no need to show data for the united kingdom as the CEO is more interested in viewing the countries that have expansion opportunities

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, date
import gspread
import gdown
from google.colab import auth
import numpy as np
import calendar
import ipywidgets as widgets
from IPython.display import display
from geopy.geocoders import Nominatim

In [ ]:


# Replace 'FILE_ID' with the actual file ID from your link
file_id = '1arBUp1j9xwdWtaFYe2HMVdlkrCkcqaZM'
url = f'https://drive.google.com/uc?id={file_id}'

# Replace 'output.csv' with the desired output file name
output_file = 'output.csv'

gdown.download(url, output_file, quiet=False)

# Load the CSV into a Pandas DataFrame
dg = pd.read_csv(output_file)

# Display the DataFrame
dg.head()



Downloading...
From: https://drive.google.com/uc?id=1arBUp1j9xwdWtaFYe2HMVdlkrCkcqaZM
To: /content/output.csv
100%|██████████| 47.2M/47.2M [00:00<00:00, 192MB/s]


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,Year,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2010,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,2010,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2010,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,2010,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,2010,3.39,17850.0,United Kingdom


In [ ]:
#Fetching information about the datasets
dg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   Year         541909 non-null  int64  
 6   UnitPrice    541909 non-null  float64
 7   CustomerID   406829 non-null  float64
 8   Country      541909 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 37.2+ MB


In [ ]:
#statistical prepresentation of the data
dg.describe()

,Quantity,Year,UnitPrice,CustomerID
count,541909.000000,541909.000000,541909.000000,406829.000000
mean,9.552250,2010.921609,4.611114,15287.690570
std,218.081158,0.268787,96.759853,1713.600303
min,-80995.000000,2010.000000,-11062.060000,12346.000000
25%,1.000000,2011.000000,1.250000,13953.000000
50%,3.000000,2011.000000,2.080000,15152.000000
75%,10.000000,2011.000000,4.130000,16791.000000
max,80995.000000,2011.000000,38970.000000,18287.000000


In [ ]:
# replace the EIRE county into USA country
dg['Country']=dg['Country'].replace('USA, USA', 'USA')
dg.loc[dg['Country'] == 'EIRE', 'Country'] = 'USA'

In [ ]:
dg['InvoiceNo'].value_counts()

573585     1114
581219      749
581492      731
580729      721
558475      705
           ... 
554023        1
554022        1
554021        1
554020        1
C558901       1
Name: InvoiceNo, Length: 25900, dtype: int64

In [ ]:
dg['StockCode'].unique()

array(['85123A', '71053', '84406B', ..., '90214U', '47591b', '23843'],
      dtype=object)

In [ ]:
dg['Description'].value_counts().head(20)

WHITE HANGING HEART T-LIGHT HOLDER    2369
REGENCY CAKESTAND 3 TIER              2200
JUMBO BAG RED RETROSPOT               2159
PARTY BUNTING                         1727
LUNCH BAG RED RETROSPOT               1638
ASSORTED COLOUR BIRD ORNAMENT         1501
SET OF 3 CAKE TINS PANTRY DESIGN      1473
PACK OF 72 RETROSPOT CAKE CASES       1385
LUNCH BAG  BLACK SKULL.               1350
NATURAL SLATE HEART CHALKBOARD        1280
POSTAGE                               1252
JUMBO BAG PINK POLKADOT               1251
HEART OF WICKER SMALL                 1237
JAM MAKING SET WITH JARS              1229
JUMBO STORAGE BAG SUKI                1214
PAPER CHAIN KIT 50'S CHRISTMAS        1210
JUMBO SHOPPER VINTAGE RED PAISLEY     1202
LUNCH BAG CARS BLUE                   1197
LUNCH BAG SPACEBOY DESIGN             1192
JAM MAKING SET PRINTED                1182
Name: Description, dtype: int64

In [ ]:
dg['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'USA', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil',
       'European Community', 'Malta', 'RSA'], dtype=object)

## Data Cleaning

In [ ]:
dg['InvoiceNo'].drop_duplicates(inplace=True)

In [ ]:

# Convert 'InvoiceNo' column to string
dg['InvoiceNo'] = dg['InvoiceNo'].astype(str)

# Remove '/C/' from the 'InvoiceNo' column
dg['InvoiceNo'] = dg['InvoiceNo'].str.replace('/C/', '', regex=True)

# Remove any remaining non-numeric characters (except for decimal points)
dg['InvoiceNo'] = dg['InvoiceNo'].str.replace('[^0-9.]', '', regex=True)

# Convert 'InvoiceNo' to numeric
dg['InvoiceNo'] = pd.to_numeric(dg['InvoiceNo'])




# Convert 'StockCode' column to string
dg['StockCode'] = dg['StockCode'].astype(str)

# Remove '/C/' from the 'StockCode' column
dg['StockCode'] = dg['StockCode'].str.replace('/C/', '', regex=True)

# Remove any remaining non-numeric characters (except for decimal points)
dg['StockCode'] = dg['StockCode'].str.replace('[^0-9.]', '', regex=True)

# Convert 'StockCode' to numeric
dg['StockCode'] = pd.to_numeric(dg['StockCode'])


In [ ]:
#getting the null values in the dataset
dg.isnull().sum()

InvoiceNo           0
StockCode        2796
Description      1454
Quantity            0
InvoiceDate         0
Year                0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [ ]:
# Filter rows where either 'Quantity' or 'UnitPrice' is less than or equal to 1
tidy_data = dg[(dg['Quantity'] <= 1) | (dg['UnitPrice'] <= 1)].index

# Drop the selected rows from the original DataFrame
dg.drop(tidy_data, inplace=True)

In [ ]:
dg.dropna(0, inplace=True)

<ipython-input-61-d33f7c5e451c>:1: FutureWarning:

In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.



In [ ]:
dg['InvoiceDate'] = pd.to_datetime(dg['InvoiceDate'])
dg['Time'] = dg['InvoiceDate'].dt.time
dg['Month'] = pd.to_datetime(dg['InvoiceDate']).dt.month
dg['Day'] = pd.to_datetime(dg['InvoiceDate']).dt.day
dg['year'] = pd.to_datetime(dg['InvoiceDate']).dt.year

# fetching the revenue column
dg['Revenue']= dg['Quantity']*dg['UnitPrice']
#fetching the sales percentage
dg['Sales_Percentage'] = (dg['UnitPrice'] / dg['UnitPrice'].sum()) * 100
#fetching the Sale Growth Percentage
dg['Sales_Growth']= dg['UnitPrice'].pct_change()*100


## what is the quantity sold per product

In [ ]:
gr=pd.DataFrame(dg.groupby(['Description'])['Quantity'].agg('sum', 'nunique').reset_index())
gr_sorted = gr.sort_values('Quantity', ascending=False).head(20)
# Create the bar chart using plotly express
fig = px.bar(data_frame=gr_sorted,x='Description',y='Quantity',color='Quantity',text='Quantity',labels={'Description': 'Product Description', 'Quantity': 'Total Quantity'},height=600)
fig.update_layout(showlegend=True,xaxis_tickangle=30,title=dict(text='Top 20 Products by Quantity sold', x=0.5), xaxis_tickfont=dict(size=10),  yaxis_tickfont=dict(size=10),margin=dict(l=10, r=20, t=50, b=20), paper_bgcolor="LightSteelblue")
fig.show()

## - The CMO is interested in viewing the top 10 countries which are generating the highest revenue. Additionallity, the CMO is also interested in viewing the quantity sold with the revenue generated. The CMO does not want to have united kingdom in the visual


In [ ]:
dg['StockCode'] = dg['StockCode'].astype(int)
gh=dg.loc[(dg['Country'] !="United Kingdom"), ["CustomerID","Revenue", "Month", "Quantity", "Year", "Country", 'StockCode']]
bv = gh.sort_values("Revenue", ascending = False).head(60).reset_index()

fig = px.choropleth(bv,
                    locations='Country',
                    locationmode='country names',
                    color='Revenue',
                    hover_name='Country',
                    projection='natural earth',
                    title='Total Sales per Country',
                    color_continuous_scale='picnic')



fig2 = px.bar(bv,
              x='Country',
              y='Quantity',
              color='Country',
              #barmode="group",
              title='the quantity sold',
             labels={'Product': 'Description', 'UnitPrice': 'Total Sales', 'Country': 'Most Popular Country'})
#fig2.add_tracer()
# Show both plots side by side
fig.show()
fig2.show()

## is there any relationship or influence of stock distribution accross country on Revenue Generation

In [ ]:
dg['StockCode'] = dg['StockCode'].astype(int)
bv = gh.sort_values("Revenue", ascending = False).reset_index()
trace=go.Heatmap(
x =bv["StockCode"],
y = bv['Country'],
z = bv['Revenue'],
type = 'heatmap',
colorscale = 'Viridis')
data=bv
fig=go.Figure(trace)
fig.show()


## - What is the monthly trend of revenue, which months have faced the biggest increase/decrease?

In [ ]:
# Group the data by year and month and calculate the total revenue for each month
monthly_revenue = dg.groupby(['year', 'Month'])['Revenue'].sum().reset_index()

# Calculate the monthly revenue changes (month-over-month changes)
monthly_revenue['Revenue_Change'] = monthly_revenue['Revenue'].diff()

# Plot the monthly revenue trend using a line chart
fig = go.Figure()

# Combine year and month columns into a single string for x-axis labeling
monthly_revenue['Month_Label'] = monthly_revenue['year'].astype(str) + '-' + monthly_revenue['Month'].astype(str)

# Add the line trace for monthly revenue
fig.add_trace(go.Scatter(x=monthly_revenue['Month_Label'], y=monthly_revenue['Revenue'],
                         mode='lines+markers',
                         name='Monthly Revenue',
                         line=dict(color='blue')))

# Add the bar trace for monthly revenue changes
fig.add_trace(go.Bar(x=monthly_revenue['Month_Label'], y=monthly_revenue['Revenue_Change'],
                     name='Monthly Revenue Change',
                     marker=dict(color='orange')))

# Update the layout for better readability
fig.update_layout(
     title=dict(text='Monthly Revenue Trend', x=0.5),  # Set the title in the center
    xaxis_title='Month',
    yaxis_title='Revenue',
    xaxis_tickangle=-45,
    legend=dict(x=0.02, y=0.98),
    margin=dict(l=10, r=20, t=50, b=20),  # Set the margin
    paper_bgcolor="LightSteelblue"  # Set the background color for the chart

)

# Show the plot
fig.show()


## the Number of Customers that reffers a Particular product Description (who are the unique Customers per product)

In [ ]:
# Group the data by product and collect the unique customers for each product
unique_customers_per_product = dg.groupby('Description')['CustomerID'].nunique().reset_index()
xc = unique_customers_per_product.head(30)
cu = xc.sort_values("CustomerID", ascending = False)
# Create a bar chart to visualize the number of unique customers per product
fig = px.bar(data_frame=cu,x='Description',y='CustomerID',color='CustomerID',text='CustomerID',labels={'Description': 'Product Description', 'CustomerID': 'Customers'},height=600)
# Update the layout
fig.update_layout(
    showlegend=True,xaxis_tickangle=30,  # Angle of the tick on x-axis
    title=dict(text='Customers Preferred Products', x=0.5),  # Set the title in the center
    xaxis_tickfont=dict(size=10),  # Set the font for x-axis
    yaxis_tickfont=dict(size=10),  # Set the font for y-axis
    margin=dict(l=10, r=20, t=50, b=20),  # Set the margin
    paper_bgcolor="LightSteelblue"  # Set the background color for the chart
)

# Show the chart
fig.show()

## Does Quantity of Goods Stocked per Country Affect Sales

In [ ]:
# Assuming 'StockCode' is a numeric column, converting it to int
dg['StockCode'] = pd.to_numeric(dg['StockCode'], errors='coerce', downcast='integer')

# Filter out rows where the "Country" is not "United Kingdom"
filtered_data = dg[dg['Country'] != "United Kingdom"]

# Get unique countries
countries = filtered_data['Country'].unique()

# Initialize geocoder
geolocator = Nominatim(user_agent='geo_plot_example')

# Fetch coordinates for each country
coordinates = []
for country in countries:
    location = geolocator.geocode(country)
    if location:
        coordinates.append((location.latitude, location.longitude))
    else:
        coordinates.append((None, None))

# Create DataFrame from the data
dg_coordinates = pd.DataFrame({"Country": countries, "Latitude": [coord[0] for coord in coordinates], "Longitude": [coord[1] for coord in coordinates]})

# Merge coordinates with the filtered data
merged_data = pd.merge(filtered_data, dg_coordinates, on='Country')

# Sort the merged DataFrame by "StockCode" in descending order
sorted_data = merged_data.sort_values("StockCode", ascending=False)

# Create scatter mapbox plot
fig = px.scatter_mapbox(
    sorted_data,
    lat="Latitude",
    lon="Longitude",
    color='Country',
    color_continuous_scale=px.colors.sequential.Blues,
    text="Country",
    title="Goods Stocked per Country",
    zoom=1,
)

fig.update_layout(
    title=dict(x=0.5),
    margin=dict(l=5, r=5, t=60, b=60),
    paper_bgcolor='lightsteelblue'
)

fig.update_traces(texttemplate="%{text}", textposition="top center")

fig.show()


## - who are the unique Customers per product

In [ ]:
ft=dg.loc[(dg['Country'] !="United Kingdom"), ["Description","CustomerID","Revenue", "Sales_Growth", "Quantity", "Year", "Country"]]
cv = ft.sort_values("Revenue", ascending = False).head(40)
fig = px.scatter(data_frame=cv, x="CustomerID", y="Revenue",  color="Revenue", size="Revenue", text="Description",
                facet_col="Country",
                 title='the unique Customers per product in Each Country', size_max=25, hover_name='CustomerID',
                log_x=False)
fig.update_layout(
    showlegend=True,
                  xaxis_tickangle=30,#angle of the tick on x-axis
                  title=dict(x=0.5), #set the title in center
                  xaxis_tickfont=dict(size=9), #set the font for x-axis
                  yaxis_tickfont=dict(size=9), #set the font for y-axis
                  margin=dict(l=10, r=20, t=50, b=20), #set the margin
                  paper_bgcolor="LightSteelblue", #set the background color for chart
)

fig.show()

##  the CEO is looking to gain insight on the demand for their products. He wants to look at the all countries and see which regions have the greatest demand for their products. once the Ceo gets an idea of the rgions that has the high demand, he will initiate an expression strategy which will allow the company to target these area and generate more business from this regions. He wants to view the entire data on no need to show data for the united kingdom as the CEO is more interested in viewing the countries that have expansion opportunities

In [ ]:

ft = dg.loc[(dg['Country'] != "United Kingdom"), ["Description", "CustomerID", "Revenue", "Sales_Growth", "Quantity", "Year", "Country"]]
cv = ft.sort_values("Revenue", ascending=False)

# Create a mapping of countries to continents
continent_mapping = {
    'United Kingdom': 'Europe',
    'Germany': 'Europe',
    'France': 'Europe',
    'EIRE': 'Europe',
    'Spain': 'Europe',
    'Netherlands': 'Europe',
    'Belgium': 'Europe',
    'Switzerland': 'Europe',
    'Portugal': 'Europe',
    'Australia': 'Oceania',
    'Norway': 'Europe',
    'Channel Islands': 'Europe',
    'Italy': 'Europe',
    'Finland': 'Europe',
    'Cyprus': 'Europe',
    'Austria': 'Europe',
    'Denmark': 'Europe',
    'Sweden': 'Europe',
    'Poland': 'Europe',
    'Japan': 'Asia',
    'Singapore': 'Asia',
    'Israel': 'Asia',
    'Iceland': 'Europe',
    'USA': 'North America',
    'Greece': 'Europe',
    'Canada': 'North America',
    'Malta': 'Europe',
    'Unspecified': 'Unknown',
    'RSA': 'Africa',
    'United Arab Emirates': 'Middle East',
    'European Community': 'Europe',
    'Lebanon': 'Middle East',
    'Lithuania': 'Europe',
    'Brazil': 'South America',
    'Bahrain': 'Middle East',
    'Czech Republic': 'Europe',
    'Saudi Arabia': 'Middle East'
}

# Add the 'Continent' column based on the mapping
cv['Continent'] = cv['Country'].map(continent_mapping)

# Create a scatter_geo plot using Plotly Express
fig = px.scatter_geo(cv, locations='Country', locationmode='country names', color='Continent',
                     size='Revenue', hover_name='Country', projection='natural earth',
                     title='Product Demand by Continent', template='plotly', width=800, height=500)

# Customize layout
fig.update_layout(title_x=0.5)

# Show the plot
fig.show()


## The Overview presentation of the dataset

In [ ]:

# Convert 'StockCode' column to integer if possible
dg['StockCode'] = pd.to_numeric(dg['StockCode'], errors='coerce')

# Filter data excluding United Kingdom
gh = dg.loc[dg['Country'] != "United Kingdom", ["CustomerID", "Description", "Revenue", "Sales_Percentage", "Sales_Growth", "Month", "Quantity", "Year", "Country", "StockCode"]]

# Sort and select top 60 rows
cv = gh.sort_values("Revenue", ascending=False).head(60)

# Create scatter matrix plot
fig = px.scatter_matrix(
    cv,
    dimensions=["Revenue", "Sales_Percentage", "Sales_Growth", "Quantity"],  # Select relevant dimensions
    color='Country',  # Color points based on country
    title='Overall detail representation of the dataset',
    width=1000,  # Adjust the width of the plot
    height=1000,  # Adjust the height of the plot
)

# Update the layout for better readability
fig.update_layout(
    title_font=dict(size=20),  # Decrease title font size for better fit
    title_x=0.5,
    title_y=0.95,
    font=dict(size=9),  # Decrease text font size for better fit
)

# Update marker properties for better visibility
fig.update_traces(marker=dict(size=5, opacity=0.7))

# Show the plot
fig.show()


## Who are the top customers and how much do they contribute to the total revenue? Is the business dependent on these customers or is the customer base diversified?

In [ ]:
# Group the data by customer and calculate the total revenue for each customer
total_revenue_per_customer = gh.sort_values("Revenue", ascending = False).reset_index()

# Sort the customers based on their total revenue in descending order to get the top customers
total_revenue_per_customer = total_revenue_per_customer.sort_values(by='Revenue', ascending=False)

# Calculate the total revenue for all customers combined
total_revenue_all_customers = total_revenue_per_customer['Revenue'].sum()

# Calculate the percentage of total revenue contributed by the top customers
total_top_customers = 0
top_customers = []
for _, row in total_revenue_per_customer.iterrows():
    total_top_customers += row['Revenue']
    top_customers.append(row['CustomerID'])
    if total_top_customers >= 0.8 * total_revenue_all_customers:
        break
percent_revenue_by_top_customers = (total_top_customers / total_revenue_all_customers) * 100

# Visualize the distribution of revenue among the top customers using a pie chart
fig = px.pie(
    total_revenue_per_customer.head(40),  # Display only the top 10 customers for better visualization
    values='Revenue',
    names='CustomerID',
    title='Top Customers Contribution to Total Revenue',
    labels={'Revenue': 'Revenue', 'CustomerID': 'CustomerID'},
    hole=0.4
)

fig.update_traces(textinfo='percent+label')  # Display percentage and label on each slice
fig.update_layout(
    showlegend=False,  # Hide legend
    margin=dict(t=50),  # Add top margin for title
    paper_bgcolor="white",  # Set background color for chart
)
# Print the top customers and their contribution to total revenue
print(f"Top Customers: {top_customers[:10]}")  # Print the IDs of the top customers
print(f"Total Revenue contributed by Top Customers: ${total_top_customers:.2f}")
print(f"Percentage of Total Revenue contributed by Top Customers: {percent_revenue_by_top_customers:.2f}%")

# Show the pie chart
fig.show()



Top Customers: [14646.0, 12755.0, 12798.0, 14646.0, 14156.0, 14156.0, 12415.0, 12798.0, 12678.0, 14646.0]
Total Revenue contributed by Top Customers: $1096851.39
Percentage of Total Revenue contributed by Top Customers: 80.00%
